## 4. Topic Modeling
Topic modeling is a text mining tool to discover semantic structures. Usually a document is described by several topics.

### 1. Latent Dirichlet Allocation
Is an statistical unsupervised model that allows sets to be explained by unobserved groups, it explains why some parts of the data are related.

#### 1. LDA in a BOW corpus
First we will create the model using a BOW corpus model created in previous steps.


In [3]:
import pickle

#LOAD LYRICS from disk
lyrics = list()
with open ('../dataset/lemma_lyrics', 'rb') as fp:
    lyrics = pickle.load(fp)

In [4]:
import gensim.corpora as corpora
from gensim.models import TfidfModel    

id2word = corpora.Dictionary(lyrics)
id2word.save("../dataset/lemma_lyrics_dict")

bow_corpus = list()

for lyric in lyrics:
    bow_corpus.append(id2word.doc2bow(lyric))

tfidf = TfidfModel(bow_corpus)
tfidf_corpus = tfidf[bow_corpus]
print(bow_corpus[0])
print(tfidf_corpus[0])

[(0, 2), (1, 1), (2, 1), (3, 13), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 2), (11, 1), (12, 1), (13, 1), (14, 3), (15, 3), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 45), (23, 1), (24, 3), (25, 3), (26, 1), (27, 5), (28, 1), (29, 6), (30, 1), (31, 1), (32, 1), (33, 4), (34, 3), (35, 1), (36, 1), (37, 1), (38, 4), (39, 1), (40, 2), (41, 1), (42, 2), (43, 2), (44, 6), (45, 1), (46, 1), (47, 2), (48, 7), (49, 1), (50, 1), (51, 1), (52, 1), (53, 1), (54, 1), (55, 3), (56, 3), (57, 1), (58, 5), (59, 2), (60, 2), (61, 3), (62, 1), (63, 1), (64, 1), (65, 3), (66, 3), (67, 2), (68, 1), (69, 1), (70, 1), (71, 2), (72, 3), (73, 2), (74, 3), (75, 1), (76, 1), (77, 1), (78, 1), (79, 1)]
[(0, 0.05693230226707338), (1, 0.027704914478609205), (2, 0.04549673720943144), (3, 0.0289448229988323), (4, 0.015806998693867935), (5, 0.024025594355196165), (6, 0.023098009819700793), (7, 0.025811526273479265), (8, 0.010569261638354028), (9, 0.006612118130594775), (10, 0.0244344778227

In [3]:
from gensim.models.ldamulticore import LdaMulticore

lda_model = LdaMulticore(workers=4,
                   corpus=bow_corpus,
                   id2word=id2word,
                   num_topics=6, 
                   #random_state=100,
                   #update_every=1,
                   #chunksize=100,
                   #passes=10,
                   per_word_topics=False)
lda_model.save("../dataset/bow_lda/lda")

#### 2. Measure the model

Coherence measure is used to distiguish bad topics from good topics. In general a bigger coherence score means a better LDA model.

It's a 4 step process:

* Segmentation: 
* Probability estimation
* Confirmation measure
* Aggregation

In [4]:
from gensim.models.coherencemodel import CoherenceModel
coherence_model_lda = CoherenceModel(model=lda_model, texts=lyrics, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Coherence Score:  0.35834682548783175


#### 3. Plot the model

In [5]:
import pyLDAvis
import pyLDAvis.gensim  

pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, bow_corpus, id2word)
vis

C:\Users\jescobedo\AppData\Local\Continuum\anaconda3\envs\ml\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3     -0.035061 -0.111136       1        1  26.117861
0      0.117223 -0.058309       2        1  18.819040
2      0.073015  0.133836       3        1  18.358023
1      0.048220 -0.021989       4        1  17.166817
4     -0.161438  0.028092       5        1  12.353893
5     -0.041959  0.029507       6        1   7.184366, topic_info=     Category          Freq     Term         Total  loglift  logprob
term                                                                
48    Default  1.077832e+06      not  1.077832e+06  30.0000  30.0000
15    Default  5.782600e+05       do  5.782600e+05  29.0000  29.0000
27    Default  7.055360e+05      get  7.055360e+05  28.0000  28.0000
97    Default  6.149250e+05       go  6.149250e+05  27.0000  27.0000
38    Default  6.010590e+05     love  6.010590e+05  26.0000  26.0000
3     Default  1.095826e+06       be  1.095826e+06  25.0000  25.0000
158   Default  2.443570e+05     baby  2.443570e+05  24.0000  24.0000
231   Default  2.691340e+05     want  2.691340e+05  23.0000  23.0000
87    Default  3.281560e+05     come  3.281560e+05  22.0000  22.0000
33    Default  5.214920e+05     know  5.214920e+05  21.0000  21.0000
44    Default  6.072300e+04    nigga  6.072300e+04  20.0000  20.0000
76    Default  9.881200e+04    wanna  9.881200e+04  19.0000  19.0000
58    Default  3.048930e+05        s  3.048930e+05  18.0000  18.0000
25    Default  5.442700e+04     fuck  5.442700e+04  17.0000  17.0000
9     Default  2.264640e+05      can  2.264640e+05  16.0000  16.0000
5118  Default  4.436800e+04    bitch  4.436800e+04  15.0000  15.0000
135   Default  1.437320e+05     girl  1.437320e+05  14.0000  14.0000
61    Default  4.658400e+04     shit  4.658400e+04  13.0000  13.0000
45    Default  3.956200e+04   niggas  3.956200e+04  12.0000  12.0000
103   Default  2.500480e+05      let  2.500480e+05  11.0000  11.0000
232   Default  1.985610e+05    would  1.985610e+05  10.0000  10.0000
29    Default  2.543200e+05     have  2.543200e+05   9.0000   9.0000
273   Default  1.759980e+05    heart  1.759980e+05   8.0000   8.0000
243   Default  1.957140e+05     good  1.957140e+05   7.0000   7.0000
141   Default  1.339950e+05     need  1.339950e+05   6.0000   6.0000
288   Default  1.168730e+05    there  1.168730e+05   5.0000   5.0000
465   Default  1.478750e+05     back  1.478750e+05   4.0000   4.0000
205   Default  7.049100e+04  believe  7.049100e+04   3.0000   3.0000
39    Default  1.467860e+05      man  1.467860e+05   2.0000   2.0000
17    Default  1.073110e+05    dream  1.073110e+05   1.0000   1.0000
...       ...           ...      ...           ...      ...      ...
400    Topic6  1.061112e+04     free  4.517287e+04   1.1847  -5.5157
205    Topic6  1.401500e+04  believe  7.049119e+04   1.0179  -5.2374
248    Topic6  1.522354e+04      lie  8.242355e+04   0.9442  -5.1547
232    Topic6  2.515789e+04    would  1.985618e+05   0.5673  -4.6524
826    Topic6  6.061377e+03    water  2.155449e+04   1.3646  -6.0756
33     Topic6  4.381792e+04     know  5.214920e+05   0.1566  -4.0975
143    Topic6  1.229920e+04      old  7.198112e+04   0.8664  -5.3680
197    Topic6  1.846938e+04    thing  1.430105e+05   0.5865  -4.9615
455    Topic6  1.057323e+04     sing  5.782863e+04   0.9341  -5.5192
243    Topic6  2.135120e+04     good  1.957142e+05   0.4177  -4.8165
288    Topic6  1.569154e+04    there  1.168740e+05   0.6253  -5.1244
60     Topic6  2.622726e+04      see  2.890638e+05   0.2334  -4.6108
39     Topic6  1.605429e+04      man  1.467865e+05   0.4203  -5.1016
188    Topic6  1.436900e+04    right  1.260067e+05   0.4620  -5.2125
97     Topic6  3.063044e+04       go  6.149251e+05  -0.3662  -4.4556
1516   Topic6  7.978847e+03     side  4.018387e+04   1.0166  -5.8008
279    Topic6  1.159338e+04  nothing  9.688739e+04   0.5101  -5.4271
358    Topic6  1.380048e+04     look  1.479987e+05   0.2608  -5.2529
162   

#### 4. LDA in Tf-idf corpus
Comparing different number of corpus we get that 26 has the biggest coherence number.

In [11]:
from gensim.models.ldamulticore import LdaMulticore
from gensim.models.coherencemodel import CoherenceModel
import warnings

warnings.filterwarnings("ignore")

best_coherence = 0.0
best_number_topics = 0
for i in range(2,30,4):
    lda_model = LdaMulticore(workers=4,
                       corpus=tfidf_corpus,
                       id2word=id2word,
                       num_topics=5, 
                       #random_state=100,
                       #update_every=1,
                       #chunksize=100,
                       #passes=10,
                       per_word_topics=False)

    coherence_model_lda = CoherenceModel(model=lda_model, texts=lyrics, dictionary=id2word, coherence='c_v')
    coherence_lda = coherence_model_lda.get_coherence()
    print('Coherence Score: '+ str(coherence_lda) + ' Topics: '+str(i))
    
    if(coherence_lda > best_coherence):
        best_coherence = coherence_lda
        best_number_topics = i
        lda_model.save("../dataset/tfidf_lda/lda")
        
print("Biggest coherence score: "+str(best_coherence)+" Number of topics: "+str(best_number_topics))

Coherence Score: 0.37556559168123016 Topics: 2
Coherence Score: 0.3812067575010482 Topics: 6
Coherence Score: 0.37322642032476006 Topics: 10
Coherence Score: 0.36373050980598454 Topics: 14
Coherence Score: 0.38201783049802585 Topics: 18
Coherence Score: 0.38521544368248006 Topics: 22
Coherence Score: 0.36888376074325324 Topics: 26
Biggest coherence score: 0.38521544368248006 Number of topics: 22


In [7]:
from gensim.models.ldamulticore import LdaMulticore
from gensim.models.coherencemodel import CoherenceModel
import warnings

warnings.filterwarnings("ignore")

lda_model = LdaMulticore(workers=4,
                       corpus=tfidf_corpus,
                       id2word=id2word,
                       num_topics=6, 
                       #random_state=100,
                       #update_every=1,
                       #chunksize=100,
                       #passes=10,
                       per_word_topics=False)
lda_model.save("../dataset/tfidf_lda/lda")
        

In [8]:
from gensim.models.ldamulticore import LdaMulticore
import pyLDAvis
import pyLDAvis.gensim  

lda_model = LdaMulticore.load("../dataset/tfidf_lda/lda")

pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, tfidf_corpus, id2word)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2     -0.149205 -0.047037       1        1  37.262611
4     -0.065711  0.140833       2        1  23.343655
1     -0.089785  0.005123       3        1  20.477564
5     -0.024406 -0.101950       4        1   9.863111
3      0.174858 -0.012595       5        1   4.565917
0      0.154249  0.015626       6        1   4.487140, topic_info=     Category         Freq       Term        Total  loglift  logprob
term                                                                
1661  Default  1034.000000   christma  1034.000000  30.0000  30.0000
38    Default  8598.000000       love  8598.000000  29.0000  29.0000
44    Default  1452.000000      nigga  1452.000000  28.0000  28.0000
17    Default  3669.000000      dream  3669.000000  27.0000  27.0000
455   Default  2261.000000       sing  2261.000000  26.0000  26.0000
436   Default  2073.000000        sky  2073.000000  25.0000  25.0000
273   Default  4542.000000      heart  4542.000000  24.0000  24.0000
45    Default  1053.000000     niggas  1053.000000  23.0000  23.0000
5118  Default  1097.000000      bitch  1097.000000  22.0000  22.0000
115   Default  2223.000000       song  2223.000000  21.0000  21.0000
110   Default  3908.000000      night  3908.000000  20.0000  20.0000
61    Default  1215.000000       shit  1215.000000  19.0000  19.0000
36    Default  1613.000000       lord  1613.000000  18.0000  18.0000
1024  Default  1578.000000       moon  1578.000000  17.0000  17.0000
286   Default  2175.000000       star  2175.000000  16.0000  16.0000
25    Default  1471.000000       fuck  1471.000000  15.0000  15.0000
257   Default  2225.000000        sun  2225.000000  14.0000  14.0000
399   Default  4266.000000        day  4266.000000  13.0000  13.0000
224   Default  1925.000000       rain  1925.000000  12.0000  12.0000
104   Default  2778.000000      light  2778.000000  11.0000  11.0000
439   Default  1287.000000      angel  1287.000000  10.0000  10.0000
1137  Default  1316.000000  beautiful  1316.000000   9.0000   9.0000
356   Default  2242.000000       kiss  2242.000000   8.0000   8.0000
421   Default  2720.000000       blue  2720.000000   7.0000   7.0000
264   Default  1099.000000     bright  1099.000000   6.0000   6.0000
212   Default  3312.000000       fall  3312.000000   5.0000   5.0000
280   Default  1419.000000       rise  1419.000000   4.0000   4.0000
158   Default  5949.000000       baby  5949.000000   3.0000   3.0000
285   Default   731.000000     spring   731.000000   2.0000   2.0000
898   Default   653.000000  wonderful   653.000000   1.0000   1.0000
...       ...          ...        ...          ...      ...      ...
4087   Topic6   158.697250       papa   274.009430   2.5578  -6.2018
6831   Topic6    79.838631         fi   103.303177   2.8463  -6.8888
2023   Topic6    56.178829      goody    67.476318   2.9207  -7.2402
1807   Topic6    98.968803      gypsy   223.674103   2.2886  -6.6740
3653   Topic6    63.052273   reindeer    86.383377   2.7891  -7.1248
898    Topic6   143.970428  wonderful   653.513306   1.5912  -6.2992
3654   Topic6    75.305305      santa   149.614197   2.4174  -6.9472
8169   Topic6    72.446922   careless   154.399963   2.3473  -6.9859
115    Topic6   153.369904       song  2223.753906   0.4299  -6.2359
38     Topic6   217.224030       love  8598.516602  -0.5745  -5.8879
3067   Topic6    74.423004     meadow   183.571930   2.2011  -6.9590
421    Topic6   130.056259       blue  2720.685059   0.0633  -6.4008
286    Topic6   120.851585       star  2175.825195   0.2134  -6.4742
508    Topic6    84.816025    embrace   385.238647   1.5906  -6.8283
615    Topic6   109.293510       many  1607.994019   0.4152  -6.5747
17     Topic6   125.582733      dream  3669.480957  -0.2709  -6.4358
87     Topic6   127.300568       come  4913.538574  -0.5492  -6.4222
205    Topic6   111.671974    believe  2866.178467  -0.1412  -6.5532
1119  

### 2. Hierarchical Dirichlet Process
HDP models topics as a mixture of words, like LDA, but the number of topics is generated by a dirichlet process. 
#### 1. Create the model

In [7]:
from gensim.models import HdpModel

hdp = HdpModel(tfidf_corpus, id2word)
hdp.save("../dataset/hdp/hdp")

In [8]:
from gensim.models import HdpModel
hdp = HdpModel.load("../dataset/hdp/hdp")
#lda_model = hdp.suggested_lda_model()
#lda_model.save("../dataset/hdp_lda/lda")

Since we didn't define a max number of topics, the model used the default 150. Here we get 6 topics and its 20 (default) most probable words. 

In [9]:
topics = []
num_topics = 6
for topic_id, topic in hdp.show_topics(num_topics=num_topics, formatted=False):
    topic = [word for word, _ in topic]
    topics.append(topic)
print(topics)

[['love', 'be', 'get', 'not', 'do', 'go', 'know', 'want', 'baby', 'have', 'let', 'time', 'never', 'would', 'can', 'ill', 'say', 'feel', 'come', 's'], ['love', 'get', 'be', 'not', 'do', 'go', 'baby', 'know', 'want', 'have', 'let', 'time', 'come', 'never', 'say', 'would', 'ill', 'can', 's', 'feel'], ['love', 'be', 'get', 'not', 'do', 'go', 'baby', 'know', 'want', 'have', 'time', 'let', 'come', 'would', 'never', 'ill', 'say', 'can', 'feel', 's'], ['love', 'be', 'get', 'not', 'do', 'go', 'baby', 'know', 'want', 'have', 'let', 'time', 'come', 'never', 'would', 'say', 'ill', 'feel', 'can', 'make'], ['love', 'be', 'get', 'not', 'do', 'go', 'baby', 'know', 'want', 'have', 'let', 'time', 'come', 'never', 'would', 'ill', 'say', 'can', 'feel', 's'], ['love', 'get', 'be', 'not', 'do', 'go', 'baby', 'know', 'want', 'have', 'time', 'let', 'come', 'never', 'ill', 'would', 'say', 'can', 's', 'feel']]


And finally the coherence of different combination of topics.

In [13]:
from gensim.models.coherencemodel import CoherenceModel

for i in range(5,100,5):
    topics = []
    num_topics = i
    for topic_id, topic in hdp.show_topics(num_topics=num_topics, formatted=False):
        topic = [word for word, _ in topic]
        topics.append(topic)
    cm = CoherenceModel(texts=lyrics, topics=topics, dictionary=id2word, coherence='c_v')
    print("Topics: ", i, " Coherence: ", cm.get_coherence())

Topics:  5  Coherence:  0.36640896081739316
Topics:  10  Coherence:  0.36636619693585215
Topics:  15  Coherence:  0.3663519423086718
Topics:  20  Coherence:  0.3663448149950816
Topics:  25  Coherence:  0.3663405386069276
Topics:  30  Coherence:  0.36633768768149166
Topics:  35  Coherence:  0.3663478695580489
Topics:  40  Coherence:  0.3663448149950816
Topics:  45  Coherence:  0.366342439223885
Topics:  50  Coherence:  0.3666439688776489
Topics:  55  Coherence:  0.3666354046033162
Topics:  60  Coherence:  0.36662826770803875
Topics:  65  Coherence:  0.36662881100427497
Topics:  70  Coherence:  0.3666782312638522
Topics:  75  Coherence:  0.36666966663692785
Topics:  80  Coherence:  0.36666217258836903
Topics:  85  Coherence:  0.366655560192582
Topics:  90  Coherence:  0.36665443632405564
Topics:  95  Coherence:  0.3666895025511239


The coherence value is the same for every value of topics, if we check the topics all of them are the same. It means the model returned only one topic.